In [1]:
import numpy as np
import pandas as pd
import random
from tqdm import tqdm

import sys
sys.path.append('/Users/andrew/Desktop/sudoku/src/sudoku')

from board import Board
from solutions import Solutions

In [2]:
def find_all_solutions(board: Board, solutions: dict) -> set:
    """
    Returns a set of Boards that contain all possible solutions to the input board
    :param board:
    :param solutions: dictionary of board K -> board V where V is the solution to K. V may be None
    :return: set of Boards
    """
#     puzzles = {board: solutions[board] for board in solutions if solutions[board]}
    if board in solutions:
        return {solutions[board]}

    found_solutions = set()
    possibilities = board.get_cell_possibilities_count()
    min_possibilities = [coord for coord in possibilities if possibilities[coord] == min(possibilities.values())]
    x, y = random.choice(min_possibilities)
    for digit in board.get_possible_digits(x, y):
        next_board = board.copy()
        next_board.write(x, y, digit)
        if next_board in solutions:
            found_solutions.add(solutions[next_board])
        elif next_board.all_filled():
            solutions[next_board] = next_board
            found_solutions.add(solutions[next_board])
        if next_board.is_solvable():
            found_solutions |= find_all_solutions(next_board, solutions)
    if len(found_solutions) == 1:
        solutions[board] = list(found_solutions)[0]
    else:
        solutions[board] = None
    return found_solutions

def generate_new_puzzle(board: Board, solutions: Solutions):
    new_puzzles = remove_cell(board, solutions, cells_to_remove=1)
    while new_puzzles:
        new_board = random.sample(new_puzzles, 1)[0]
        new_puzzles = remove_cell(new_board, solutions, cells_to_remove=1)
    return new_board if new_board != board else None

def remove_cell(board: Board, solutions: Solutions, cells_to_remove: int=0) -> list:
    """
    Given a board with a unique solution and a dictionary of board-hash -> solution-board-hash, remove a random cell
     from the board and test if the resulting board also has a unique solution (every legal move also results boards
     that have unique solutions). Also adds the new board to unique_solutions
    If no board with a unique solution can be created, returns None
    :param board:
    :param solutions:
    :param cells_to_remove:
    :return:
    """
    assert board in solutions
    assert cells_to_remove >= 0
    new_puzzles = []
    xs, ys = np.nonzero(board.board)
    indices = np.arange(len(xs))
    np.random.shuffle(indices)
    for i in indices:
        x, y = xs[i], ys[i]
        new_board = board.remove(x, y)
        if new_board in solutions: # if this puzzle already exists, skip
            pass
        if len(find_all_solutions(new_board, solutions)) == 1:
            new_puzzles.append(new_board)
            if cells_to_remove and len(new_puzzles) >= cells_to_remove:
                return new_puzzles
    return new_puzzles if new_puzzles else None

In [3]:
solutions = Solutions()
solutions.load('../data/solutions2.txt')

In [5]:
solutions = Solutions()
board = Board(2, 2)
for i in range(4):
    board.write(0, i, i+1)
find_all_solutions(board, solutions)

{array([[1, 2, 3, 4],
        [3, 4, 1, 2],
        [2, 1, 4, 3],
        [4, 3, 2, 1]], dtype=int8), array([[1, 2, 3, 4],
        [3, 4, 1, 2],
        [2, 3, 4, 1],
        [4, 1, 2, 3]], dtype=int8), array([[1, 2, 3, 4],
        [3, 4, 1, 2],
        [4, 1, 2, 3],
        [2, 3, 4, 1]], dtype=int8), array([[1, 2, 3, 4],
        [3, 4, 1, 2],
        [4, 3, 2, 1],
        [2, 1, 4, 3]], dtype=int8), array([[1, 2, 3, 4],
        [3, 4, 2, 1],
        [2, 1, 4, 3],
        [4, 3, 1, 2]], dtype=int8), array([[1, 2, 3, 4],
        [3, 4, 2, 1],
        [4, 3, 1, 2],
        [2, 1, 4, 3]], dtype=int8), array([[1, 2, 3, 4],
        [4, 3, 1, 2],
        [2, 1, 4, 3],
        [3, 4, 2, 1]], dtype=int8), array([[1, 2, 3, 4],
        [4, 3, 1, 2],
        [3, 4, 2, 1],
        [2, 1, 4, 3]], dtype=int8), array([[1, 2, 3, 4],
        [4, 3, 2, 1],
        [2, 1, 4, 3],
        [3, 4, 1, 2]], dtype=int8), array([[1, 2, 3, 4],
        [4, 3, 2, 1],
        [2, 4, 1, 3],
        [3, 1, 4, 2]], dt

In [6]:
solutions.save('../data/solutions.txt')

In [7]:
a = solutions.get_random_seed_solution()
for i in range(8):
    a = remove_cell(a, solutions, cells_to_remove=1)[0]

In [8]:
sorted(solutions.seed_solutions)

[array([[1, 2, 3, 4],
        [3, 4, 1, 2],
        [2, 1, 4, 3],
        [4, 3, 2, 1]], dtype=int8), array([[1, 2, 3, 4],
        [3, 4, 1, 2],
        [2, 3, 4, 1],
        [4, 1, 2, 3]], dtype=int8), array([[1, 2, 3, 4],
        [3, 4, 1, 2],
        [4, 1, 2, 3],
        [2, 3, 4, 1]], dtype=int8), array([[1, 2, 3, 4],
        [3, 4, 1, 2],
        [4, 3, 2, 1],
        [2, 1, 4, 3]], dtype=int8), array([[1, 2, 3, 4],
        [3, 4, 2, 1],
        [2, 1, 4, 3],
        [4, 3, 1, 2]], dtype=int8), array([[1, 2, 3, 4],
        [3, 4, 2, 1],
        [4, 3, 1, 2],
        [2, 1, 4, 3]], dtype=int8), array([[1, 2, 3, 4],
        [4, 3, 1, 2],
        [2, 1, 4, 3],
        [3, 4, 2, 1]], dtype=int8), array([[1, 2, 3, 4],
        [4, 3, 1, 2],
        [3, 4, 2, 1],
        [2, 1, 4, 3]], dtype=int8), array([[1, 2, 3, 4],
        [4, 3, 2, 1],
        [2, 1, 4, 3],
        [3, 4, 1, 2]], dtype=int8), array([[1, 2, 3, 4],
        [4, 3, 2, 1],
        [2, 4, 1, 3],
        [3, 1, 4, 2]], dt

In [33]:
print(len(solutions))
for i in tqdm(range(10)):
    sol = solutions.get_random_seed_solution(inverse_weight=True)
#     print(sol)
    generate_new_puzzle(sol, solutions)
#     print(len(solutions))

  0%|          | 0/10 [00:00<?, ?it/s]

24185


100%|██████████| 10/10 [00:15<00:00,  1.57s/it]


In [30]:
len(solutions)

24185

In [34]:
solutions.save('../data/solutions5.txt')

In [32]:
for board, sol in tqdm([(b, s) for b, s in solutions.items() if s]):
    if sol and board != sol:
        all_sols = list(find_all_solutions(board, Solutions()))
        assert len(all_sols) is 1 and all_sols[0] == sol

100%|██████████| 13712/13712 [00:49<00:00, 274.55it/s]


In [ ]:
for board, sol in tqdm([(b, s) for b, s in solutions.items() if not s]):
    assert len(find_all_solutions(board, Solutions())) != 1

 78%|███████▊  | 8418/10745 [03:02<00:50, 46.07it/s]

In [7]:
solutions.get_random_seed_solution(inverse_weight=True)

array([[1, 2, 3, 4],
       [4, 3, 2, 1],
       [3, 1, 4, 2],
       [2, 4, 1, 3]], dtype=int8)

In [6]:
np.arange(len(list(solutions.seed_solutions)))

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152])

In [7]:
solutions.seed_solutions

{array([[1, 2, 3, 4],
        [3, 4, 1, 2],
        [2, 1, 4, 3],
        [4, 3, 2, 1]], dtype=int8), array([[1, 2, 3, 4],
        [3, 4, 1, 2],
        [2, 3, 4, 1],
        [4, 1, 2, 3]], dtype=int8), array([[1, 2, 3, 4],
        [3, 4, 1, 2],
        [4, 1, 2, 3],
        [2, 3, 4, 1]], dtype=int8), array([[1, 2, 3, 4],
        [3, 4, 1, 2],
        [4, 3, 2, 1],
        [2, 1, 4, 3]], dtype=int8), array([[1, 2, 3, 4],
        [3, 4, 2, 1],
        [2, 1, 4, 3],
        [4, 3, 1, 2]], dtype=int8), array([[1, 2, 3, 4],
        [3, 4, 2, 1],
        [4, 3, 1, 2],
        [2, 1, 4, 3]], dtype=int8), array([[1, 2, 3, 4],
        [4, 3, 1, 2],
        [2, 1, 4, 3],
        [3, 4, 2, 1]], dtype=int8), array([[1, 2, 3, 4],
        [4, 3, 1, 2],
        [3, 4, 2, 1],
        [2, 1, 4, 3]], dtype=int8), array([[1, 2, 3, 4],
        [4, 3, 2, 1],
        [2, 1, 4, 3],
        [3, 4, 1, 2]], dtype=int8), array([[1, 2, 3, 4],
        [4, 3, 2, 1],
        [2, 4, 1, 3],
        [3, 1, 4, 2]], dt

In [7]:
np.random.choice(np.arange(5))

4